--- Day 14: Parabolic Reflector Dish ---

You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that lets you tilt it in one of four directions! The rounded rocks (O) will roll when the platform is tilted, while the cube-shaped rocks (#) will stay in place. You note the positions of all of the empty spaces (.) and rocks (your puzzle input). For example:

```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```

Start by tilting the lever so all of the rocks will slide north as far as they will go:

```
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
```

You notice that the support beams along the north side of the platform are damaged; to ensure the platform doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (O) is equal to the number of rows from the rock to the south edge of the platform, including the row the rock is on. (Cube-shaped rocks (#) don't contribute to load.) So, the amount of load caused by each rock in each row is as follows:

```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```

The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is 136.

Tilt the platform so that the rounded rocks all roll north. Afterward, what is the total load on the north support beams?

Your puzzle answer was 109596.

In [ ]:
import numpy as np

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()
    
def read_platform(text):
    return [[char for char in line] for line in text.strip().splitlines()]

def print_platform(platform):
    for line in platform:
        print(''.join(line))

def find_free_spot_north(platform, start):
    spot = start
    (sx, sy) = (start[0], start[1] - 1)
    while (0 <= sy < len(platform) and platform[sy][sx] == '.'):
        spot = (sx, sy)
        sy -= 1
    return spot

def tilt_platform_north(platform):
    for (y, rocks) in enumerate(platform):
        for (x, rock) in enumerate(rocks):
            if rock == 'O':
                (nx, ny) = find_free_spot_north(platform, (x, y))
                platform[y][x] = '.'
                platform[ny][nx] = 'O'
    return platform;

def tilt_platform_east(platform):
    return np.transpose(tilt_platform_south(np.transpose(platform)))

def tilt_platform_south(platform):
    return tilt_platform_north(platform[::-1])[::-1]

def tilt_platform_west(platform):
    return np.transpose(tilt_platform_north(np.transpose(platform)))

def calculate_load(platform):
    return sum([sum([len(platform) - y for rock in rocks if rock == 'O']) for (y, rocks) in enumerate(platform)])

input = read_input('sample.txt')
platform = read_platform(input)
tilted_north = tilt_platform_north(np.array(platform, copy=True))

print_platform(tilted_north)
print('-----------------')
print(calculate_load(tilted_north))

--- Part Two ---

The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:

After 1 cycle:

```
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....
```

After 2 cycles:

```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O
```

After 3 cycles:

```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
```

This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams after 1000000000 cycles.

In the above example, after 1000000000 cycles, the total load on the north support beams is 64.

Run the spin cycle for 1000000000 cycles. Afterward, what is the total load on the north support beams?

Your puzzle answer was 96105.

In [ ]:
import hashlib

def cycle_platform(platform, cycles):
    cycled = platform
    for _ in range(cycles):
        cycled = tilt_platform_north(cycled)
        cycled = tilt_platform_west(cycled)
        cycled = tilt_platform_south(cycled)
        cycled = tilt_platform_east(cycled)
        yield cycled

def find_repeated_cycle(platform, cycles):
    history = []
    for (index, cycled) in enumerate(cycle_platform(platform, cycles)):
        cycled_string = ''.join([''.join(row) for row in cycled])
        checksum = hashlib.sha384(cycled_string.encode()).hexdigest()
        if checksum in history:
            return (history.index(checksum), index - history.index(checksum))
        history.append(checksum)

input = read_input('sample.txt')
platform = read_platform(input)

(cycle_start, cycle_length) = find_repeated_cycle(np.array(platform, copy=True), 10000)
cycle_overflow = 1000000000 % cycle_length + cycle_length * 10
print(f'Cycle starts at {cycle_start} and has length {cycle_length}, use overflow {cycle_overflow} to calculate final platform')

cycled_platform = [cycle for cycle in cycle_platform(np.array(platform, copy=True), cycle_overflow)][-1]
print_platform(cycled_platform)
print('-----------------')
print(calculate_load(cycled_platform))
